First let's make sure that we import all the required libraries to parse the wikipedia webpage, extract the table from it and populate it into the pandas dataframe.

In [31]:
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json
import ssl
import pandas as pd

Then we need to define the url of the webpage that we will parse, then parse it, extract the table, define column names, rows and prepare a dataframe which we will be populating with the information from the table on the webpage.

In [32]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html,'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody
rows = table.find_all('tr')
columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]

df = pd.DataFrame(columns = columns)
df_hoods = df

Once the dataframe is ready we need to create a loop which will iterate through the lines of html code that we have extracted using BeautifulSoup. We also need to create an "if" condition to ignore the rows where 'Borough' cell is equl to 'Not assigned'. If it is not then we populate our dataframe with values from the row.

In [33]:
for i in range(1,len(rows)):
    tds = rows[i].find_all('td')
    
    if tds[1].text == 'Not assigned':
        continue
    else:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', '')]
        df_hoods = df_hoods.append(pd.Series(values, index = columns), ignore_index = True)
df_hoods.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Further we check whether 'Not assigned' value still exist in the dataframe in both 'Borough' and 'Neighbourhood' columns.

In [34]:
print('Not assigned' in df_hoods['Borough'].unique())
print('Not assigned' in df_hoods['Neighbourhood'].unique())

False
True


So as it is seen from the results of the previous cell's code there are still 'Not assigned' values in the 'Neighbourhood' column. Therefore, we need to replace those values with the corresponding 'Borough' values. I have created a loop to iterate through values in the 'Neighbourhood' column and if 'Not assigned' is found replaced it with the value in the corresponding 'Borough' column.

In [35]:
for i in range(0,len(df_hoods['Neighbourhood'])):
    if df_hoods['Neighbourhood'].iloc[i] == 'Not assigned':
        df_hoods['Neighbourhood'].iloc[i] = df_hoods['Borough'].iloc[i]
df_hoods.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Now lets check one more time if 'Not assigned' value exists in any of the two columns.

In [36]:
print('Not assigned' in df_hoods['Borough'].unique())
print('Not assigned' in df_hoods['Neighbourhood'].unique())

False
False


Now that we have removed all 'Not assigned' values, there is still one more step to do to clean out the data. As you may have noticed there might be several neighbourhoods in the same postalcode area. We need to group such neighbourhoods into a single row and list all the neighbourhoods in the single cell separated by coma. I have applied groupby attribute of the pandas dataframes along with applying formating to separate neighbourhood names by coma.

In [95]:
df_clean = df_hoods.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_clean.rename(columns = {'Postcode':'PostalCode'}, inplace = True)
df_clean.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Finally, as it was requested in the requirements to this assignment we need to check the shape of the clean dataframe that we have obtained from wikipedia.

In [38]:
df_clean.shape

(103, 3)

Next I need to populate the content of csv file with location coordinates into the new pandas dataframe and rename the 'Postal Code' column into 'PostalCode' so that we can merge two dataframes by matching on this column.

In [89]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
df_geo.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Once I have downloaded new dataframe with coordinates I need to merge these two dataframes as follows:

In [93]:
df_location = df_clean.merge(df_geo, how = 'left', on = 'PostalCode')
df_location.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Let's check the shape of the final dataframe to make sure that I did not append any unnece

In [94]:
df_location.shape

(103, 5)